In [31]:

import numpy as np


D,H,W = 256 , 640, 640
padding, kernel_size, stride = 0, 3, 1

depth_weight_size = D*kernel_size*kernel_size
w = np.zeros(depth_weight_size)
for i in range(0, depth_weight_size):
    die = np.random.uniform()
    if die < 1/3:
        w[i] = -1
    if die > 2/3:
        w[i] = 1
weightfilename = 'w.txt'
with open(weightfilename, 'w') as f:
    for i in range(0, depth_weight_size):
        f.write(f'{int(w[i])}\n')

we = np.reshape(w, (D, kernel_size, kernel_size))



v2 = f''
for d in range(0, D):
    f = f'fs = {d*H*W};\n'
    f += f'for (int i = 0; i < Hout; i++){{\n'
    f += f'\tfor (int j = 0; j < Wout; j++){{\n'
    f += f'\t\tO[j + i*Wout + {d}*Wout*Hout] += '
    for h in range(0, kernel_size):
        for w in range(0, kernel_size):
            if we[d, h, w] == -1:
                f += f'+I[fs + {w + h*W}]'
            if we[d, h, w] == 1:
                f += f'-I[fs + {w + h*W}]'
    f += f'+0;\n'
    f += f'\t\tfs += stride;\n'
    f += f'\t}}\n'
    f += f'\tfs = stride*Win + {d*H*W};\n'
    f += f'}}\n'
    v2 += f
    
v3 = f''
for d in range(0, D):
    f = f'fs = {d*H*W};\n'
    f += f'for (int i = 0; i < Hout; i++){{\n'
    f += f'\tfor (int j = 0; j < Wout; j++){{\n'
    f += f'\t\tO[(j+i*Wout)*Dout + {d}] += '
    for h in range(0, kernel_size):
        for w in range(0, kernel_size):
            if we[d, h, w] == -1:
                f += f'+I[fs + {w + h*W}]'
            if we[d, h, w] == 1:
                f += f'-I[fs + {w + h*W}]'
    f += f'+0;\n'
    f += f'\t\tfs += stride;\n'
    f += f'\t}}\n'
    f += f'\tfs = stride*Win + {d*H*W};\n'
    f += f'}}\n'
    v3 += f


In [32]:
zah = f'''
#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>
#include <immintrin.h>

#define wfilename "{weightfilename}"
#define Din {D}
#define Hin {H}
#define Win {W}
#define kernel_size {kernel_size}
#define stride {stride}
#define INARRLEN Din*Hin*Win
#define Dout Din
#define MAXFILTERDEPTH Din/8
#define Hout (Hin - kernel_size)/stride + 1 
#define Wout (Win - kernel_size)/stride + 1 
#define OUTARRLEN Dout*Hout*Wout

int A[INARRLEN];
int B[OUTARRLEN];
int C[OUTARRLEN];
int D[OUTARRLEN];
int E[OUTARRLEN];
int F[OUTARRLEN];
int G[OUTARRLEN];
int W[Din*kernel_size*kernel_size];






void transpose(int I[], int O[], int M, int N){{
    for (int i = 0; i < M; i++){{
        for (int j = 0; j < N; j++){{
            O[i + j*M] = I[j + i*N];
        }}
    }}
}}




void conv2d_depth_transposed_subfilter(int I[], int O[], int W[]){{
    memset(O, 0, OUTARRLEN*sizeof(int));
    int cur_subfilter[Din];
    for (int h = 0; h < kernel_size; h++){{
        for (int w = 0; w < kernel_size; w++){{
            int fs = w*Din + h*kernel_size*Din;
            memcpy(cur_subfilter, &W[w*Din + h*kernel_size*Din], Din*sizeof(int));
            for (int i = 0; i < Hout; i++){{
                fs = Win*stride*Din + w*Din + h*kernel_size*Din;
                for (int j = 0; j < Wout; j++){{
                    for (int d = 0; d < Din; d++){{
                        O[j*Din + i*Wout*Din + d] += I[fs + d] * cur_subfilter[d];
                    }}
                    fs += Din*stride;
                }}
                
            }}
            
        }}
    }}
}}

void conv2d_depth_transposed_hypervectorized(int I[], int O[], int W[]){{

    __m256i filter00[Din/8];
    __m256i filter01[Din/8];
    __m256i filter02[Din/8];
    __m256i filter10[Din/8];
    __m256i filter11[Din/8];
    __m256i filter12[Din/8];
    __m256i filter20[Din/8];
    __m256i filter21[Din/8];
    __m256i filter22[Din/8];
    __m256i cur00[Din/8];
    __m256i cur01[Din/8];
    __m256i cur02[Din/8];
    __m256i cur10[Din/8];
    __m256i cur11[Din/8];
    __m256i cur12[Din/8];
    __m256i cur20[Din/8];
    __m256i cur21[Din/8];
    __m256i cur22[Din/8];
    
    for (int n = 0; n < Din/8; n++){{filter00[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*0]);}}
    for (int n = 0; n < Din/8; n++){{filter01[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*1]);}}
    for (int n = 0; n < Din/8; n++){{filter02[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*2]);}}
    for (int n = 0; n < Din/8; n++){{filter10[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*3]);}}
    for (int n = 0; n < Din/8; n++){{filter11[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*4]);}}
    for (int n = 0; n < Din/8; n++){{filter12[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*5]);}}
    for (int n = 0; n < Din/8; n++){{filter20[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*6]);}}
    for (int n = 0; n < Din/8; n++){{filter21[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*7]);}}
    for (int n = 0; n < Din/8; n++){{filter22[n] = _mm256_loadu_si256((__m256i*)&W[n*8 + Din*8]);}}
    

    for (int i = 0; i < Hout; i++){{
        for (int j = 0; j < Wout; j++){{
            int fs = Din*(j*stride + i*stride*Win);
            for (int n = 0; n < Din/8; n++){{cur00[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*0 + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur01[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*1 + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur02[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*2 + fs]);}}

            for (int n = 0; n < Din/8; n++){{cur10[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*0 + 1*Wout*Din + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur11[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*1 + 1*Wout*Din  + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur12[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*2 + 1*Wout*Din  + fs]);}}

            for (int n = 0; n < Din/8; n++){{cur20[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*0 + 2*Wout*Din  + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur21[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*1 + 2*Wout*Din  + fs]);}}
            for (int n = 0; n < Din/8; n++){{cur22[n] = _mm256_loadu_si256((__m256i*)&I[n*8 + Din*2 + 2*Wout*Din  + fs]);}}
            
            for (int n = 0; n < Din/8; n++){{cur00[n] = _mm256_mullo_epi32(cur00[n], filter00[n]);}}
            for (int n = 0; n < Din/8; n++){{cur01[n] = _mm256_mullo_epi32(cur00[n], filter01[n]);}}
            for (int n = 0; n < Din/8; n++){{cur02[n] = _mm256_mullo_epi32(cur00[n], filter02[n]);}}
            for (int n = 0; n < Din/8; n++){{cur10[n] = _mm256_mullo_epi32(cur00[n], filter10[n]);}}
            for (int n = 0; n < Din/8; n++){{cur11[n] = _mm256_mullo_epi32(cur00[n], filter11[n]);}}
            for (int n = 0; n < Din/8; n++){{cur12[n] = _mm256_mullo_epi32(cur00[n], filter12[n]);}}
            for (int n = 0; n < Din/8; n++){{cur20[n] = _mm256_mullo_epi32(cur00[n], filter20[n]);}}
            for (int n = 0; n < Din/8; n++){{cur21[n] = _mm256_mullo_epi32(cur00[n], filter21[n]);}}
            for (int n = 0; n < Din/8; n++){{cur22[n] = _mm256_mullo_epi32(cur00[n], filter22[n]);}}

            for (int n = 0; n < Din/8; n++){{
                cur00[n] = _mm256_add_epi32(cur00[n], cur01[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur02[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur10[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur11[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur12[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur20[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur21[n]);
                cur00[n] = _mm256_add_epi32(cur00[n], cur22[n]);
            }}
            for (int n = 0; n < Din/8; n++){{
                _mm256_storeu_si256((__m256i*)&O[(j + i*Wout)*Din + n*8] , cur00[n]);
            }}
        }}
    }}
}}









int main(){{
    for (long int i = 0; i < INARRLEN; i++){{A[i] = rand()%512 - 256;}}
    FILE* wfile = fopen(wfilename, "r");
    for (int i = 0; i < Din*kernel_size*kernel_size; i++){{fscanf(wfile, "%d\\n", &W[i]);}}
    fclose(wfile);
    int max;
    
    


    clock_t time_vbelovedpts_beg = clock();
    conv2d_depth_transposed_subfilter(A, F, W);
    clock_t time_vbelovedpts_end = clock();
    printf("time for conv2d depth version transposed and subfiltered: %f s\\n", (double)(time_vbelovedpts_end - time_vbelovedpts_beg)/CLOCKS_PER_SEC);
    max = F[0];for (int i = 1; i < OUTARRLEN; i++){{if (F[i] > max){{max = F[i];}}}}
    FILE* garbagevbelovedipts = fopen("GBG/garbagevbelovedptts.gbg", "w");fprintf(garbagevbelovedipts, "%d", max);fclose(garbagevbelovedipts);

    clock_t time_vbelovedptshv_beg = clock();
    conv2d_depth_transposed_hypervectorized(A, G, W);
    clock_t time_vbelovedptshv_end = clock();
    printf("time for conv2d depth version transposed and hypervectorized: %f s\\n", (double)(time_vbelovedptshv_end - time_vbelovedptshv_beg)/CLOCKS_PER_SEC);
    max = G[0];for (int i = 1; i < OUTARRLEN; i++){{if (G[i] > max){{max = G[i];}}}}
    FILE* garbagevbelovediptshv = fopen("GBG/garbagevbelovedpttshv.gbg", "w");fprintf(garbagevbelovediptshv, "%d", max);fclose(garbagevbelovediptshv);

    return 0;
}}
'''

ccode = zah



with open("depth_code.c", "w") as f:
    f.write(ccode)

!cd /home/oleksiy-tsuber/finding_best_multithreaded_conv
!gcc -g -march=native -O3 -o depth_code depth_code.c
print("compilation done")
!./depth_code

depth_code.c: In function ‘main’:
depth_code.c:154:59: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
  154 |     for (int i = 0; i < Din*kernel_size*kernel_size; i++){fscanf(wfile, "%d\n", &W[i]);}
      |                                                           ^~~~~~~~~~~~~~~~~~~~~~~~~~~~
compilation done
time for conv2d depth version transposed and subfiltered: 0.074522 s
time for conv2d depth version transposed and hypervectorized: 0.115265 s


In [33]:
3**4

81